# Unetのセグメンテーションモデル

In [1]:
pip install segmentation_models_pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=5d85bfa58c88da7bd88f23bc9ac72e8801c77f94ae0462ce1f1812d2fe98a2f6
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=be5ed8e49ad2c6622f562f3e4ad97984faa72bb1c629224ef6b94e8893e891ee
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch

In [2]:
pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00


In [3]:
pip install timm

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pydicom
import numpy as np
import os
import glob
from tqdm import tqdm
import gc

import torchvision
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from fastai.vision.all import *
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
CV = 5
SEED = 777
fold = 1
PATCH_SIZE = 512
patch_size = 64
TH = .5
SEG_TRAIN = False
SEG = {
    'BS':16,
    'LR':5e-4,
    'EPOCHS':10
}
INF = {
    'BS':64,
    'LR':1e-5,
    'EPOCHS':20
}

In [7]:
!unzip /content/drive/MyDrive/rsna-2024-lumbar-spine-degenerative-classification.zip -d /content #RSNA2024を今のディレクトリに展開する

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: /content/train_images/885894528/2509467535/43.dcm  
  inflating: /content/train_images/885894528/2509467535/44.dcm  
  inflating: /content/train_images/885894528/2509467535/45.dcm  
  inflating: /content/train_images/885894528/2509467535/46.dcm  
  inflating: /content/train_images/885894528/2509467535/47.dcm  
  inflating: /content/train_images/885894528/2509467535/48.dcm  
  inflating: /content/train_images/885894528/2509467535/49.dcm  
  inflating: /content/train_images/885894528/2509467535/5.dcm  
  inflating: /content/train_images/885894528/2509467535/50.dcm  
  inflating: /content/train_images/885894528/2509467535/6.dcm  
  inflating: /content/train_images/885894528/2509467535/7.dcm  
  inflating: /content/train_images/885894528/2509467535/8.dcm  
  inflating: /content/train_images/885894528/2509467535/9.dcm  
  inflating: /content/train_images/885894528/3303042519/1.dcm  
  inflating: /content/train_images/885894528/3303042519/10.dcm  

In [8]:
train = pd.read_csv('/content/train.csv')
train.tail()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
1970,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1971,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1972,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1973,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild
1974,4290709089,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [9]:
diagnosis = list(filter(lambda x: x.find('foraminal') > -1, train.columns))
train = train[train[diagnosis].isnull().values.sum(1)==0].reset_index(drop=True)
train.tail()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
1962,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1963,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1964,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1965,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild
1966,4290709089,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [10]:
labels = {
    'Normal/Mild':0,
    'Moderate':1,
    'Severe':2
}

In [11]:
df_meta_f = pd.read_csv('/content/drive/MyDrive/RSNA_csv/train_series_descriptions.csv')
df_meta_f.tail()

,study_id,series_id,series_description
6289,4287160193,1507070277,Sagittal T2/STIR
6290,4287160193,1820446240,Axial T2
6291,4290709089,3274612423,Sagittal T2/STIR
6292,4290709089,3390218084,Axial T2
6293,4290709089,4237840455,Sagittal T1


In [12]:
df_meta_f.rename(columns={'series_description':'series_description_2'},inplace=True)

In [13]:
df_coor = pd.read_csv('/content/train_label_coordinates.csv')
df_coor.tail()

,study_id,series_id,instance_number,condition,level,x,y
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333
48691,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L5/S1,211.813953,259.534884


In [14]:
merged_df = pd.merge(df_coor, df_meta_f, on=['study_id', 'series_id'])

In [15]:
SS=merged_df[merged_df['condition'].isin(['Right Subarticular Stenosis','Left Subarticular Stenosis'])]

In [16]:
# 新しいカラムを初期化


# study_id, series_id, instance_numberごとにグループ化して処理
for (study_id, series_id, instance_number), group in SS.groupby(['study_id', 'series_id', 'instance_number']):
    # 'Left Subarticular Stenosis' の場合、該当する座標を取得
    left_condition = group[group['condition'] == 'Left Subarticular Stenosis']
    if not left_condition.empty:
        # 対応する座標を取得し、グループに値を一度に設定
        SS.loc[(SS['study_id'] == study_id) & (SS['series_id'] == series_id) & (SS['instance_number'] == instance_number), ['x_left', 'y_left']] = \
            left_condition[['x', 'y']].values[0]

    # 'Right Subarticular Stenosis' の場合、該当する座標を取得
    right_condition = group[group['condition'] == 'Right Subarticular Stenosis']
    if not right_condition.empty:
        # 対応する座標を取得し、グループに値を一度に設定
        SS.loc[(SS['study_id'] == study_id) & (SS['series_id'] == series_id) & (SS['instance_number'] == instance_number), ['x_right', 'y_right']] = \
            right_condition[['x', 'y']].values[0]

# 確認
print(SS.head())


    study_id   series_id  instance_number                    condition  level  \
15   4003253  2448190387                3   Left Subarticular Stenosis  L1/L2   
16   4003253  2448190387                4  Right Subarticular Stenosis  L1/L2   
17   4003253  2448190387               11   Left Subarticular Stenosis  L2/L3   
18   4003253  2448190387               11  Right Subarticular Stenosis  L2/L3   
19   4003253  2448190387               19   Left Subarticular Stenosis  L3/L4   

             x           y series_description_2      x_left      y_left  \
15  179.126448  161.235521             Axial T2  179.126448  161.235521   
16  145.288771  158.624642             Axial T2         NaN         NaN   
17  180.979730  158.764479             Axial T2  180.979730  158.764479   
18  145.900042  157.096466             Axial T2  180.979730  158.764479   
19  176.037645  157.528958             Axial T2  176.037645  157.528958   

       x_right     y_right  
15         NaN         NaN  
16  

In [17]:
SS2=SS.dropna()

In [18]:
SCS=merged_df[merged_df['condition']=='Spinal Canal Stenosis']

In [19]:
F=df_coor[df_coor['condition'].isin(['Left Subarticular Stenosis','Right Subarticular Stenosis'])][[
    'study_id',
    'series_id',
    'instance_number',
    'level',
    'condition',
    'x',
    'y'
]].sort_values([
    'study_id',
    'series_id',
    'instance_number'
])[[
    'study_id',
    'series_id',
    'level',
    'instance_number',
    'condition',
    'x',
    'y'
]]
F.tail()

,study_id,series_id,level,instance_number,condition,x,y
48677,4290709089,3390218084,L3/L4,10,Right Subarticular Stenosis,302.875911,356.304937
48678,4290709089,3390218084,L4/L5,15,Left Subarticular Stenosis,348.203719,341.369400
48679,4290709089,3390218084,L4/L5,15,Right Subarticular Stenosis,305.745866,340.520184
48680,4290709089,3390218084,L5/S1,20,Left Subarticular Stenosis,355.777684,368.960270
48681,4290709089,3390218084,L5/S1,21,Right Subarticular Stenosis,302.875911,364.627811


In [20]:
target = F.columns[-6:-1]
target

Index(['series_id', 'level', 'instance_number', 'condition', 'x'], dtype='object')

In [21]:
labels = {
    'Normal/Mild':0,
    'Moderate':1,
    'Severe':2
}

coor = [
    'x_left',
    'y_left',
    'x_right',
    'y_right'
]

In [22]:
target2=['subarticular_stenosis_l1_l2', 'subarticular_stenosis_l2_l3',
       'subarticular_stenosis_l3_l4', 'subarticular_stenosis_l4_l5',
       'subarticular_stenosis_l5_s1']

In [23]:
def augment_image_and_centers(image,centers,alpha):
    '''
    # Randomly flip the image horizontally.
    if random.random() > .5:
      if random.random() > 1 - alpha:
        image = image.flip(-1)
        centers[:,0] = PATCH_SIZE - centers[:,0]
    # Randomly flip the image vertically.
    if random.random() > 0.5:
      if random.random() > 1 - alpha:
        image = image.flip(-2)
        centers[:,1] = PATCH_SIZE - centers[:,1]

    if random.random() > 1 - alpha:
      if random.random() > .5:
    #   Randomly flip the image
    #   Wich axis?
         axis = np.random.randint(2)
        image = image.flip(axis+1)
        centers[:,-1-axis] = PATCH_SIZE - centers[:,-1-axis]
    '''
#   Randomly rotate the image.
    angle = torch.as_tensor(random.uniform(-180, 180)*alpha)
    image = torchvision.transforms.functional.rotate(image,angle.item())
#   https://discuss.pytorch.org/t/rotation-matrix/128260
    angle = angle*math.pi/180
    s = torch.sin(angle)
    c = torch.cos(angle)
    rot = torch.stack([
        torch.stack([c, s]),
        torch.stack([-s, c])
      ])
    centers = ((centers.cpu() - PATCH_SIZE//2) @ rot) + PATCH_SIZE//2
    return image,centers

In [24]:
class T1Dataset(Dataset):
    def __init__(self, df, VALID=False, alpha=0):
        self.data = df
        self.VALID = VALID
        self.alpha = alpha

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]

        centers = torch.as_tensor([x for x in row[coor]]).view(2,2).float()

        sample = '/content/drive/MyDrive/dataset/'
        sample = sample+str(row['study_id'])+'/'+str(row['series_id'])+'/'+str(row['instance_number'])+'.dcm'
        image = pydicom.dcmread(sample).pixel_array
        H,W = image.shape
#       By plane resizing I've been distorting the proportions
        if H > W:
            d = W
            if not self.VALID:
                h = int((H - d)*(.5 + self.alpha*(.5 - np.random.rand())))
            else:
                h = (H - d)//2
            image = image[h:h+d]
            centers[:,1] -= h
            H = W
        elif H < W:
            d = H
            if not self.VALID:
                w = int((W - d)*(.5 + self.alpha*(.5 - np.random.rand())))
            else:
                w = (W - d)//2
            image = image[:,w:w+d]
            centers[:,0] -= w
            W = H
        image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
        image = torch.as_tensor(image/np.max(image)).unsqueeze(0).float()

        label = torch.as_tensor(1)

        centers[:,0] = centers[:,0]*PATCH_SIZE/W
        centers[:,1] = centers[:,1]*PATCH_SIZE/H

        if not self.VALID: image,centers = augment_image_and_centers(image,centers,self.alpha)
        return image.to(device),[label.to(device),centers.to(device)]

In [25]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [26]:
class myUNet2(nn.Module):
    def __init__(self):
        super(myUNet2, self).__init__()

        self.UNet = smp.Unet(
            encoder_name="resnet18",
            classes=2,
            in_channels=1
        ).to(device)

    def forward(self,X):
        x = self.UNet(X)
#       MinMaxScaling along the class plane to generate a heatmap
        min_values = x.view(-1,2,PATCH_SIZE*PATCH_SIZE).min(-1)[0].view(-1,2,1,1) # Bug, I've been MinMaxScaling with the wrong values
        max_values = x.view(-1,2,PATCH_SIZE*PATCH_SIZE).max(-1)[0].view(-1,2,1,1)
        x = (x - min_values)/(max_values - min_values)

        return x

In [27]:
idx_map = torch.stack([torch.arange(PATCH_SIZE)]*PATCH_SIZE).to(device)
idx_map = torch.stack([idx_map,idx_map.T]).view(1,1,2,PATCH_SIZE,PATCH_SIZE)
class myLoss2(nn.Module):
    def __init__(
            self,
            alpha=.5
        ):
        super().__init__()
        self.alpha = alpha

    def clone(self):
        return myLoss(self.alpha)

    def forward(
            self,
            y,# Predictions
            t # Targets
        ):
        mask_pred = y
        _,mask_true = t
#       The heatmap Loss as the distance between the predicted Normal and the ideal one
#       Let's define the ideal heatmaps as the Normal distributions
#       centered on the diagnostic centers with s2 = PATCH_SIZE/8
        s2 = s2 = torch.as_tensor([PATCH_SIZE/8]*2)
#       Then the corresponding alphas and normalization constants would be
        A = -1/(2*s2).to(device)
        K = 1/torch.sqrt(2*math.pi*s2).to(device)
#       Predicted heatmaps rescaling
        mask_pred = mask_pred*K.view(1,2,1,1)
#       Ideal heatmaps
        mask = idx_map - mask_true.view(-1,2,2,1,1)
        mask = torch.exp((A.view(-1,2,1,1,1)*mask*mask).sum(2))*K.view(-1,2,1,1)
#       Distance
        D = 1 - ((mask*mask_pred).sum())**2/((mask*mask).sum()*(mask_pred*mask_pred).sum())

        return D


In [28]:
import numpy as np

# SS2の行数を取得
n_rows = len(SS2)

# 1から5の値を繰り返して作成し、行数に合わせる
series_description_values = np.tile([1, 2, 3, 4, 5], n_rows // 5 + 1)[:n_rows]

# 新しいカラム 'series_description' に値を代入（locを使用して直接設定）
SS2 = SS2.copy()  # 明示的にコピーを作成して変更を加える
SS2.loc[:, 'series_description'] = series_description_values

# 結果を確認
print(SS2.head())


    study_id   series_id  instance_number                    condition  level  \
17   4003253  2448190387               11   Left Subarticular Stenosis  L2/L3   
18   4003253  2448190387               11  Right Subarticular Stenosis  L2/L3   
19   4003253  2448190387               19   Left Subarticular Stenosis  L3/L4   
20   4003253  2448190387               19  Right Subarticular Stenosis  L3/L4   
21   4003253  2448190387               28   Left Subarticular Stenosis  L4/L5   

             x           y series_description_2      x_left      y_left  \
17  180.979730  158.764479             Axial T2  180.979730  158.764479   
18  145.900042  157.096466             Axial T2  180.979730  158.764479   
19  176.037645  157.528958             Axial T2  176.037645  157.528958   
20  142.843690  156.179561             Axial T2  176.037645  157.528958   
21  172.948842  156.911197             Axial T2  172.948842  156.911197   

       x_right     y_right  series_description  
17  145.90004

In [29]:
SEG_TRAIN=True

In [30]:
import pickle

tdf = SS2[SS2['series_description'] != fold]
vdf = SS2[SS2['series_description'] == fold]



tds = T1Dataset(tdf)
vds = T1Dataset(vdf,VALID=True)
tdl = torch.utils.data.DataLoader(tds, batch_size=SEG['BS'], shuffle=True, drop_last=True)
vdl = torch.utils.data.DataLoader(vds, batch_size=SEG['BS'], shuffle=False)

if SEG_TRAIN:
    seed_everything(SEED)

    dls = DataLoaders(tdl,vdl)

    n_iter = len(tds)//SEG['BS']

    model = myUNet2()
    learn = Learner(
        dls,
        model,
        lr=SEG['LR'],
        loss_func=myLoss2(alpha=0.5),
        # cbs=[
        #     ShowGraphCallback(),
        #     alpha_cb
        # ]
    )
    learn.fit_one_cycle(SEG['EPOCHS'])
    with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+str(fold)+"_SS"+".pkl", 'wb') as f:
      pickle.dump(model, f)
    del tdl,vdl,dls,model,learn
    gc.collect()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 252MB/s]


epoch,train_loss,valid_loss,time


KeyboardInterrupt: 

# axialT2画像の前処理

In [38]:
import os
import pandas as pd

# データセットのパス
base_dir = '/content/train_images/'

# リストを初期化
data = []

# 一番上の階層（study_id）のフォルダをたどる
for study_id in os.listdir(base_dir):
    study_path = os.path.join(base_dir, study_id)
    if os.path.isdir(study_path):
        # 二番目の階層（series_id）のフォルダをたどる
        for series_id in os.listdir(study_path):
            series_path = os.path.join(study_path, series_id)
            if os.path.isdir(series_path):
                # 三番目の階層（〇〇.dcm ファイル）をたどる
                for filename in os.listdir(series_path):
                    if filename.endswith('.dcm'):
                        # 〇〇.dcm の〇〇部分（instance_number）を抽出
                        instance_number = filename.split('.')[0]
                        # データを追加
                        data.append([study_id, series_id, instance_number])

# pandas DataFrameを作成
df = pd.DataFrame(data, columns=['study_id', 'series_id', 'instance_number'])

# DataFrameの表示
print(df)

          study_id   series_id instance_number
0       3668205319  3292993552               1
1       3668205319  3292993552               5
2       3668205319  3292993552              16
3       3668205319  3292993552              18
4       3668205319  3292993552               6
...            ...         ...             ...
147213  1575827384   354713324              11
147214  1575827384   354713324               8
147215  1575827384   354713324               4
147216  1575827384   354713324              12
147217  1575827384   354713324               7

[147218 rows x 3 columns]


In [39]:
# new_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')

df_meta_f = pd.read_csv('/content/train_series_descriptions.csv')
df_meta_f.tail()

df = df.astype('int64')

all_df = pd.merge(df, df_meta_f, on=['study_id', 'series_id'], how='inner')

s_all_df=all_df[all_df['series_description']=='Sagittal T1']

a_all_df=all_df[all_df['series_description']=='Axial T2']

s2_all_df=all_df[all_df['series_description']=='Sagittal T2/STIR']

In [40]:
import os
import numpy as np

# データフレームをコピーして操作
a_all_df_copy = a_all_df.copy()

# x_pos カラムを NaN で初期化
a_all_df_copy['x_pos'] = None
a_all_df_copy['y_pos'] = None
a_all_df_copy['z_pos'] = None
a_all_df_copy['pixel_sp_z'] = None
a_all_df_copy['pixel_sp_y'] = None

for idx, row in a_all_df_copy.iterrows():
    # DICOMファイルのパスを構築
    dicom_file_path = f"/content/train_images/{row['study_id']}/{row['series_id']}/{row['instance_number']}.dcm"

    # DICOMファイルを読み込む
    dicom_data = pydicom.dcmread(dicom_file_path)
    H,W = dicom_data.pixel_array.shape


    # 'Image Position (Patient)' の x 座標を取得して x_pos カラムに格納
    a_all_df_copy.loc[idx, 'x_pos'] = dicom_data.ImagePositionPatient[0]
    a_all_df_copy.loc[idx, 'y_pos'] = dicom_data.ImagePositionPatient[1]
    a_all_df_copy.loc[idx, 'z_pos'] = dicom_data.ImagePositionPatient[2]
    a_all_df_copy.loc[idx, 'pixel_sp_z']=dicom_data.PixelSpacing[0]
    a_all_df_copy.loc[idx, 'pixel_sp_y']=dicom_data.PixelSpacing[1]
    a_all_df_copy.loc[idx, 'H'] = H
    a_all_df_copy.loc[idx, 'W'] = W
    a_all_df_copy.loc[idx, 'mean_bfr_cut_y'] = H/2
    a_all_df_copy.loc[idx, 'mean_bfr_cut_x'] = W/2
    a_all_df_copy.loc[idx,'o0']=dicom_data.ImageOrientationPatient[0]
    a_all_df_copy.loc[idx,'o1']=dicom_data.ImageOrientationPatient[1]
    a_all_df_copy.loc[idx,'o2']=dicom_data.ImageOrientationPatient[2]
    a_all_df_copy.loc[idx,'o3']=dicom_data.ImageOrientationPatient[3]
    a_all_df_copy.loc[idx,'o4']=dicom_data.ImageOrientationPatient[4]
    a_all_df_copy.loc[idx,'o5']=dicom_data.ImageOrientationPatient[5]

print(a_all_df_copy)

          study_id   series_id  instance_number series_description    x_pos  \
0       3668205319  3292993552                1           Axial T2 -114.237   
1       3668205319  3292993552                5           Axial T2 -114.342   
2       3668205319  3292993552               16           Axial T2 -114.697   
3       3668205319  3292993552               18           Axial T2 -114.844   
4       3668205319  3292993552                6           Axial T2 -114.247   
...            ...         ...              ...                ...      ...   
147201  1575827384  3977611865                4           Axial T2 -100.621   
147202  1575827384  3977611865               27           Axial T2 -100.035   
147203  1575827384  3977611865               24           Axial T2 -99.4183   
147204  1575827384  3977611865               12           Axial T2 -96.9498   
147205  1575827384  3977611865                7           Axial T2 -100.603   

          y_pos    z_pos pixel_sp_z pixel_sp_y     

In [41]:
a_all_df_copy = a_all_df_copy.reset_index(drop=True)


a_all_df=a_all_df_copy


In [42]:
with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+"1"+"_SS"+".pkl", 'rb') as f:
      model2=pickle.load(f)

In [43]:
import torch

# (OUT > 0.5)[0][1] がテンソルマスクだと仮定
a_all_df_copy = a_all_df.copy()
for j in range(len(a_all_df)):

  # a_all_df[['study_id', 'series_id', 'instance_number']] = a_all_df[['study_id', 'series_id', 'instance_number']].astype(int)
  image=pydicom.dcmread("/content/train_images/"+str(a_all_df_copy.iloc[j]["study_id"])+"/"+str(a_all_df_copy.iloc[j]["series_id"])+"/"+str(a_all_df_copy.iloc[j]["instance_number"])+".dcm").pixel_array
  dicom=pydicom.dcmread("/content/train_images/"+str(a_all_df_copy.iloc[j]["study_id"])+"/"+str(a_all_df_copy.iloc[j]["series_id"])+"/"+str(a_all_df_copy.iloc[j]["instance_number"])+".dcm")
  H,W = image.shape
  a_all_df_copy.loc[j, 'H'] = H
  a_all_df_copy.loc[j, 'W'] = W
  a_all_df_copy.loc[j,'o0']=dicom.ImageOrientationPatient[0]
  a_all_df_copy.loc[j,'o1']=dicom.ImageOrientationPatient[1]
  a_all_df_copy.loc[j,'o2']=dicom.ImageOrientationPatient[2]
  a_all_df_copy.loc[j,'o3']=dicom.ImageOrientationPatient[3]
  a_all_df_copy.loc[j,'o4']=dicom.ImageOrientationPatient[4]
  a_all_df_copy.loc[j,'o5']=dicom.ImageOrientationPatient[5]
  a_all_df_copy.loc[j, 'H_cut'] = 0
  a_all_df_copy.loc[j, 'W_cut'] = 0
  if H > W:
      d = W
      h = (H - d)//2
      image = image[h:h+d]
      a_all_df_copy.loc[j, 'H_cut'] = h
      # centers[:,1] -= h
      H = W
  elif H < W:
      d = H
      w = (W - d)//2
      image = image[:,w:w+d]
      a_all_df_copy.loc[j, 'W_cut'] = w
      # centers[:,0] -= w
      W = H
  image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
  image = torch.as_tensor(image/np.max(image)).unsqueeze(0).float()
  image=image.to(device)
  OUT = model2(image.unsqueeze(0)).cpu().detach()
  for i in range(2):
    mask = (OUT > 0.5)[0][i]  # 512x512のテンソル

    # Trueのピクセル座標を取得（y座標とx座標のペアとして取得）
    true_pixels = torch.nonzero(mask)

    # 平均座標を計算
    mean_yx = true_pixels.float().mean(dim=0)

    # 平均座標を表示
    mean_y, mean_x = mean_yx[0], mean_yx[1]


    a_all_df_copy.loc[j, f'mean_y_{i}']=mean_y.item()
    a_all_df_copy.loc[j, f'mean_x_{i}']=mean_x.item()
    print(f"Mean coordinates: (y, x) = ({0-mean_y.item()}, {mean_x.item()})")


ストリーミング出力は最後の 5000 行に切り捨てられました。
Mean coordinates: (y, x) = (-255.37681579589844, 274.26666259765625)
Mean coordinates: (y, x) = (-253.64285278320312, 205.903564453125)
Mean coordinates: (y, x) = (-270.0886535644531, 273.9326171875)
Mean coordinates: (y, x) = (-268.1654968261719, 230.9683074951172)
Mean coordinates: (y, x) = (-269.721435546875, 276.0785827636719)
Mean coordinates: (y, x) = (-267.52728271484375, 228.6436309814453)
Mean coordinates: (y, x) = (-268.8021545410156, 269.5395812988281)
Mean coordinates: (y, x) = (-267.73834228515625, 228.10752868652344)
Mean coordinates: (y, x) = (-268.0848083496094, 270.06005859375)
Mean coordinates: (y, x) = (-265.79998779296875, 232.23928833007812)
Mean coordinates: (y, x) = (-263.2562255859375, 273.98931884765625)
Mean coordinates: (y, x) = (-262.7978210449219, 226.95668029785156)
Mean coordinates: (y, x) = (-268.0249938964844, 275.6856994628906)
Mean coordinates: (y, x) = (-265.6462097167969, 230.88809204101562)
Mean coordinates: (y, x) =

In [44]:
a_all_df=a_all_df_copy

import torch

# 各マスクの座標を元の画像サイズに戻すコード
for j in range(len(a_all_df)):
    # 元の画像サイズ
    H = a_all_df.loc[j, 'H']
    W = a_all_df.loc[j, 'W']

    # 512にリサイズされた画像のサイズ
    resized_size = 512

    # 高さと幅のスケール（リサイズ前の座標に戻すためのスケール）
    if H<W:
      scale_y = H / resized_size
      scale_x = H / resized_size
    elif H>W:
      scale_y = W / resized_size
      scale_x = W / resized_size
    else:
      scale_y = W / resized_size
      scale_x = H / resized_size
    for i in range(2):
        # リサイズ後の座標を取得
        mean_y_resized =a_all_df.loc[j, f'mean_y_{i}']
        mean_x_resized =a_all_df.loc[j, f'mean_x_{i}']

        # リサイズ前の座標に戻す（スケールを掛ける）
        mean_bfr_y = mean_y_resized * scale_y
        mean_bfr_x = mean_x_resized * scale_x

        # 戻した座標を新しいカラムに格納
        a_all_df.loc[j, f'mean_bfr_y_{i}'] = mean_bfr_y
        a_all_df.loc[j, f'mean_bfr_x_{i}'] = mean_bfr_x


In [45]:
import torch

# 各マスクの切り取る前の座標を計算して新しいカラムに格納
for j in range(len(a_all_df)):
    # H_cut, W_cut はどれくらい切り取られたかを表す（NaNの場合は切り取りなし）
    H_cut = a_all_df.loc[j, 'H_cut'] if not pd.isna(a_all_df.loc[j, 'H_cut']) else 0
    W_cut = a_all_df.loc[j, 'W_cut'] if not pd.isna(a_all_df.loc[j, 'W_cut']) else 0

    for i in range(2):
        # リサイズ前の平均座標を取得
        mean_bfr_y = a_all_df.loc[j, f'mean_bfr_y_{i}']
        mean_bfr_x = a_all_df.loc[j, f'mean_bfr_x_{i}']

        # 切り取る前の座標を計算（H_cutとW_cutを戻す）
        mean_bfr_cut_y = mean_bfr_y + H_cut
        mean_bfr_cut_x = mean_bfr_x + W_cut

        # 切り取る前の座標を新しいカラムに格納
        a_all_df.loc[j, f'mean_bfr_cut_y_{i}'] = mean_bfr_cut_y
        a_all_df.loc[j, f'mean_bfr_cut_x_{i}'] = mean_bfr_cut_x

In [46]:
import numpy as np

# 3次元座標を計算してカラムに格納するコード
for j in range(len(a_all_df)):
    # DICOMのメタデータ
    x_pos = a_all_df.loc[j, 'x_pos']
    y_pos = a_all_df.loc[j, 'y_pos']
    z_pos = a_all_df.loc[j, 'z_pos']

    o0, o1, o2 = a_all_df.loc[j, ['o0', 'o1', 'o2']]  # 行方向ベクトル (ImageOrientationPatientの0〜2番目)
    o3, o4, o5 = a_all_df.loc[j, ['o3', 'o4', 'o5']]  # 列方向ベクトル (ImageOrientationPatientの3〜5番目)

    pixel_sp_y = a_all_df.loc[j, 'pixel_sp_y']  # 行方向のPixelSpacing
    pixel_sp_z = a_all_df.loc[j, 'pixel_sp_z']  # 列方向のPixelSpacing

    # 行方向ベクトル
    row_direction = np.array([o0, o1, o2])
    # 列方向ベクトル
    col_direction = np.array([o3, o4, o5])
    for i in range(2):
        # ピクセル座標を取得
        mean_y = a_all_df.loc[j, f'mean_bfr_cut_y_{i}']  # 行方向（R）
        mean_x = a_all_df.loc[j, f'mean_bfr_cut_x_{i}']  # 列方向（C）

        # 3次元座標を計算
        patient_pos = np.array([x_pos, y_pos, z_pos]) + \
                      (pixel_sp_y * mean_x) * row_direction + \
                      (pixel_sp_z * mean_y) * col_direction

        # 計算した3次元座標をカラムに格納
        a_all_df.loc[j, f'xx_{i}'] = patient_pos[0]
        a_all_df.loc[j, f'yy_{i}'] = patient_pos[1]
        a_all_df.loc[j, f'zz_{i}'] = patient_pos[2]


In [47]:
a_all_df = a_all_df.rename(columns={
    'xx_0': 'xx_0_axial',
    'yy_0': 'yy_0_axial',
    'zz_0': 'zz_0_axial',
    'xx_1': 'xx_1_axial',
    'yy_1': 'yy_1_axial',
    'zz_1': 'zz_1_axial'
})

In [48]:
with open('/content/drive/MyDrive/RSNA_csv/middle_s_all_df_rev.pkl', 'rb') as f:
  middle_s_all_df=pickle.load(f)

In [49]:
import pandas as pd

# middle_s_all_df の z_pos カラムを study_id をキーにして a_all_df に結合
a_all_df_2 = pd.merge(a_all_df, middle_s_all_df[['study_id', 'zz_0', 'zz_1', 'zz_2', 'zz_3', 'zz_4','z_pos_0','z_pos_1','z_pos_2','z_pos_3','z_pos_4']], on='study_id', how='left')

# 結果を表示
print(a_all_df_2.head())

     study_id   series_id  instance_number series_description    x_pos  \
0  3668205319  3292993552                1           Axial T2 -114.237   
1  3668205319  3292993552                5           Axial T2 -114.342   
2  3668205319  3292993552               16           Axial T2 -114.697   
3  3668205319  3292993552               18           Axial T2 -114.844   
4  3668205319  3292993552                6           Axial T2 -114.247   

     y_pos    z_pos pixel_sp_z pixel_sp_y      H  ...        zz_0        zz_1  \
0 -31.2222  168.519   0.520833   0.520833  384.0  ...  148.242283  118.504717   
1  -43.086  132.894   0.520833   0.520833  384.0  ...  148.242283  118.504717   
2 -38.1561 -10.4045   0.520833   0.520833  384.0  ...  148.242283  118.504717   
3 -34.7968 -18.5367   0.520833   0.520833  384.0  ...  148.242283  118.504717   
4  -43.865  128.565   0.520833   0.520833  384.0  ...  148.242283  118.504717   

        zz_2       zz_3       zz_4     z_pos_0    z_pos_1    z_pos_2

In [50]:
import numpy as np

# levelを設定する関数
def assign_level(row):
    # 各z_posの範囲を計算（絶対値で範囲を確保）

    z0_mid = (row['zz_1'] - row['zz_0']) / 2
    z1_mid = (row['zz_2'] - row['zz_1']) / 2
    z2_mid = (row['zz_3'] - row['zz_2']) / 2
    z3_mid = (row['zz_4'] - row['zz_3']) / 2
    z4_mid = z3_mid  # z_pos_4の範囲はz_pos_3と同じ幅を使用

    bool1=(row['zz_4'] - row['zz_3']>=0)
    bool2=(row['zz_3'] - row['zz_2']>=0)
    bool3=(row['zz_2'] - row['zz_1']>=0)
    bool4=(row['zz_1'] - row['zz_0']>=0)
    if ((bool1+bool2+bool3+bool4)>=2):
      if (row['zz_1'] - z1_mid >= row['zz_0_axial']) and (row['zz_0'] - z1_mid <= row['zz_0_axial']):
          return 'L1/L2'
      elif (row['zz_1'] - z1_mid <= row['zz_0_axial']) and (row['zz_0_axial']<= row['zz_1'] + z1_mid):
          return 'L2/L3'
      elif (row['zz_1'] + z1_mid <= row['zz_0_axial']) and (row['zz_0_axial'] <= row['zz_2'] + z2_mid):
          return 'L3/L4'
      elif (row['zz_2'] + z2_mid <= row['zz_0_axial']) and (row['zz_0_axial'] <= row['zz_3'] + z3_mid):
          return 'L4/L5'
      elif (row['zz_0_axial'] >=  row['zz_3'] + z3_mid) and (row['zz_0_axial'] <=  row['zz_4'] + z3_mid):
          return 'L5/S1'
      else:
          return 'OUT'  # 該当しない場合にはNaNを返す
    else:
      if (row['zz_1'] - z1_mid <= row['zz_0_axial']) and (row['zz_0'] - z1_mid >= row['zz_0_axial']):
          return 'L1/L2'
      elif (row['zz_1'] - z1_mid >= row['zz_0_axial']) and (row['zz_0_axial'] >= row['zz_1'] + z1_mid):
          return 'L2/L3'
      elif (row['zz_1'] + z1_mid >= row['zz_0_axial']) and (row['zz_0_axial'] >= row['zz_2'] + z2_mid):
          return 'L3/L4'
      elif (row['zz_2'] + z2_mid >= row['zz_0_axial']) and (row['zz_0_axial'] >= row['zz_3'] + z3_mid):
          return 'L4/L5'
      elif (row['zz_0_axial'] <= row['zz_3'] + z3_mid) and (row['zz_0_axial'] >= row['zz_4'] + z3_mid):
          return 'L5/S1'
      else:
          return 'OUT'  # 該当しない場合にはNaNを返す

a_all_df_2['level'] = a_all_df_2.apply(assign_level, axis=1)

# 結果を確認
print(a_all_df_2[['zz_0_axial', 'level']].head())


   zz_0_axial  level
0  148.597977  L1/L2
1  115.437558  L2/L3
2   27.063492  L5/S1
3   18.621206  L5/S1
4  111.205416  L2/L3


In [51]:
import pandas as pd
import numpy as np

# 関数：グループごとに均等にデータを選び、足りない場合は補完
def select_five_from_group(group):
    group_sorted = group.sort_values(by='zz_0_axial').reset_index(drop=True)
    num_rows = len(group_sorted)

    if num_rows >= 5:
        # 均等に5つ選ぶ
        step = num_rows / 5
        selected_indices = [int(i * step) for i in range(5)]
        return group_sorted.iloc[selected_indices]

    elif num_rows == 4:
        # 4つの場合：最後のデータを追加
        selected_indices = list(range(4)) + [3]
        return group_sorted.iloc[selected_indices]
    elif num_rows == 3:
        # 3つの場合：最後の2つと1つを追加
        selected_indices = [0, 1, 2, 2, 1]
        return group_sorted.iloc[selected_indices]

    elif num_rows == 2:
        # 2つの場合：最後の1つを追加
        selected_indices = [0, 1, 1, 1, 0]
        return group_sorted.iloc[selected_indices]

    elif num_rows == 1:
        # 1つの場合：そのデータを5回繰り返す
        return pd.concat([group_sorted] * 5, ignore_index=True)


def process_dataframe(df):
    grouped_df = df.groupby(['study_id', 'series_id', 'level'], group_keys=False)
    result_df = grouped_df.apply(select_five_from_group)
    return result_df.reset_index(drop=True)

# a_all_df_2 を読み込んで処理
# a_all_df_2 = pd.read_csv('your_data.csv')  # データを読み込む部分
sampled_df = process_dataframe(a_all_df_2)

# 結果を確認
print(sampled_df)



         study_id   series_id  instance_number series_description      x_pos  \
0         4003253  2448190387                5           Axial T2  -95.31121   
1         4003253  2448190387                4           Axial T2  -95.28412   
2         4003253  2448190387                3           Axial T2  -95.25703   
3         4003253  2448190387                2           Axial T2 -95.229935   
4         4003253  2448190387                1           Axial T2  -95.20284   
...           ...         ...              ...                ...        ...   
62055  4290709089  3390218084               23           Axial T2 -97.204577   
62056  4290709089  3390218084               22           Axial T2 -97.141781   
62057  4290709089  3390218084               21           Axial T2 -97.078985   
62058  4290709089  3390218084               20           Axial T2 -97.016189   
62059  4290709089  3390218084               19           Axial T2 -96.953393   

           y_pos       z_pos pixel_sp_z

<ipython-input-51-adf25fcbb8c4>:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = grouped_df.apply(select_five_from_group)


In [52]:
merged_df = pd.merge(sampled_df,train, on='study_id', how='left')

In [68]:
import pandas as pd

# まず、merged_dfのコピーを作成して2倍にする
merged_df_left = merged_df.copy()
merged_df_left['direction'] = 'left'

merged_df_right = merged_df.copy()
merged_df_right['direction'] = 'right'

# 2つのデータフレームを縦方向に結合して行を倍にする
merged_df_doubled = pd.concat([merged_df_left, merged_df_right], ignore_index=True)

# 結果を表示
print(merged_df_doubled)

          study_id   series_id  instance_number series_description      x_pos  \
0          4003253  2448190387                5           Axial T2  -95.31121   
1          4003253  2448190387                4           Axial T2  -95.28412   
2          4003253  2448190387                3           Axial T2  -95.25703   
3          4003253  2448190387                2           Axial T2 -95.229935   
4          4003253  2448190387                1           Axial T2  -95.20284   
...            ...         ...              ...                ...        ...   
124115  4290709089  3390218084               23           Axial T2 -97.204577   
124116  4290709089  3390218084               22           Axial T2 -97.141781   
124117  4290709089  3390218084               21           Axial T2 -97.078985   
124118  4290709089  3390218084               20           Axial T2 -97.016189   
124119  4290709089  3390218084               19           Axial T2 -96.953393   

            y_pos       z_p

In [69]:
merged_df_doubled['direction'] = merged_df_doubled['direction'].replace({'left': 0, 'right': 1})

<ipython-input-69-1c472f0cba1d>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df_doubled['direction'] = merged_df_doubled['direction'].replace({'left': 0, 'right': 1})


In [70]:
# ラベルを設定するための関数
def assign_label(row):
    if row['direction'] == 0:
        if row['level'] == 'L1/L2':
            return row['left_subarticular_stenosis_l1_l2']
        elif row['level'] == 'L2/L3':
            return row['left_subarticular_stenosis_l2_l3']
        elif row['level'] == 'L3/L4':
            return row['left_subarticular_stenosis_l3_l4']
        elif row['level'] == 'L4/L5':
            return row['left_subarticular_stenosis_l4_l5']
        elif row['level'] == 'L5/S1':
            return row['left_subarticular_stenosis_l5_s1']
    elif row['direction'] == 1:
        if row['level'] == 'L1/L2':
            return row['right_subarticular_stenosis_l1_l2']
        elif row['level'] == 'L2/L3':
            return row['right_subarticular_stenosis_l2_l3']
        elif row['level'] == 'L3/L4':
            return row['right_subarticular_stenosis_l3_l4']
        elif row['level'] == 'L4/L5':
            return row['right_subarticular_stenosis_l4_l5']
        elif row['level'] == 'L5/S1':
            return row['right_subarticular_stenosis_l5_s1']
    return None

# apply関数を使ってlabelカラムを作成
merged_df_doubled['label'] = merged_df_doubled.apply(assign_label, axis=1)

In [71]:
def group_rows(df, group_size):
    grouped = []
    num_groups = len(df) // group_size + int(len(df) % group_size != 0)

    for i in range(num_groups):
        start_idx = i * group_size
        end_idx = min((i + 1) * group_size, len(df))
        subset = df.iloc[start_idx:end_idx].reset_index(drop=True)

        # グループ内の先頭行の値を取得
        first_row = subset.iloc[0]

        # 新しいカラムに instance_number を格納
        row_data = first_row.to_dict()  # 他の列の情報も含めて最初の行の情報を取得
        row_data.update({
            f'instance_number_{j}': subset.iloc[j]['instance_number'] for j in range(len(subset))
        })

        # フィルの中身をリストに格納
        grouped.append(row_data)

    return pd.DataFrame(grouped)

# グループサイズを設定（5つの行を1行にまとめる）
group_size = 5
grouped_df_axial = group_rows(merged_df_doubled, group_size)
print(grouped_df_axial.head())

   study_id   series_id  instance_number series_description      x_pos  \
0   4003253  2448190387                5           Axial T2 -95.311210   
1   4003253  2448190387               13           Axial T2 -95.527960   
2   4003253  2448190387               21           Axial T2 -95.744710   
3   4003253  2448190387               30           Axial T2 -98.615036   
4   4003253  2448190387               36           Axial T2 -97.249850   

       y_pos      z_pos  pixel_sp_z  pixel_sp_y      H  ...  \
0 -26.274843 -392.73752       0.625       0.625  320.0  ...   
1 -26.884478 -427.93158       0.625       0.625  320.0  ...   
2 -27.494120 -463.12564       0.625       0.625  320.0  ...   
3 -15.932791 -525.72110       0.625       0.625  320.0  ...   
4  -8.653456 -551.06100       0.625       0.625  320.0  ...   

   right_subarticular_stenosis_l3_l4  right_subarticular_stenosis_l4_l5  \
0                        Normal/Mild                        Normal/Mild   
1                        N

In [72]:
grouped_df_axial=grouped_df_axial[grouped_df_axial['level']!='OUT']

In [73]:
# Split grouped_df_axial by 'study_id' and 'direction', then remove duplicate 'level' rows
def remove_duplicates(grouped_df_axial):
    # Group by 'study_id' and 'direction'
    grouped = grouped_df_axial.groupby(['study_id', 'direction'])

    # Remove duplicates in each group based on 'level'
    unique_groups = []
    for _, group in grouped:
        # Drop duplicates based on the 'level' column
        unique_group = group.drop_duplicates(subset='level')
        unique_groups.append(unique_group)

    # Concatenate the results back into a single DataFrame
    return pd.concat(unique_groups, ignore_index=True)

# Apply the function to grouped_df_axial
grouped_df_axial = remove_duplicates(grouped_df_axial)

In [74]:
import pandas as pd

# Define all possible levels
all_levels = ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']

# Function to fill missing levels
def fill_missing_levels(grouped_df_axial, all_df):
    # Create a new dataframe to store filled rows
    filled_rows = []

    # Iterate over each combination of study_id, series_id
    for (study_id,direction), group in grouped_df_axial.groupby(['study_id','direction']):
        existing_levels = group['level'].tolist()

        # Find missing levels
        missing_levels = [lvl for lvl in all_levels if lvl not in existing_levels]
        row = group.iloc[0].copy()
        for i in range(len(group)):
          row['series_id'] = group.iloc[i]['series_id']
          series_id = group.iloc[i]['series_id']
          all_group = all_df[(all_df['study_id'] == study_id) & (all_df['series_id'] == series_id)]
          instance_numbers = all_group['instance_number'].tolist()
          total_len = len(instance_numbers)
          if total_len > 1:
            series_id = group.iloc[i]['series_id']
            row['series_id'] = group.iloc[i]['series_id']
            break
        # Fetch the row from all_df with matching study_id and series_id
        all_group = all_df[(all_df['study_id'] == study_id) & (all_df['series_id'] == series_id)]
        instance_numbers = all_group['instance_number'].tolist()
        total_len = len(instance_numbers)
        for missing_level in missing_levels:
            # Fetch the row from all_df with matching study_id and series_id
            row = group.iloc[0].copy()

            # Assign the missing level
            row['level'] = missing_level

            # Calculate instance_number based on missing level
            if missing_level == 'L1/L2':
                row['instance_number_0'] = instance_numbers[max(total_len // 5,1)]
                row['instance_number_1'] = instance_numbers[max(total_len // 5,1)]
                row['instance_number_2'] = instance_numbers[max(total_len // 5,1)]
                row['instance_number_3'] = instance_numbers[max(total_len // 5,1)]
                row['instance_number_4'] = instance_numbers[max(total_len // 5,1)]
            elif missing_level == 'L2/L3':
                row['instance_number_0'] = instance_numbers[max((total_len * 2) // 5,1)]
                row['instance_number_1'] = instance_numbers[max((total_len * 2) // 5,1)]
                row['instance_number_2'] = instance_numbers[max((total_len * 2) // 5,1)]
                row['instance_number_3'] = instance_numbers[max((total_len * 2) // 5,1)]
                row['instance_number_4'] = instance_numbers[max((total_len * 2) // 5,1)]
            elif missing_level == 'L3/L4':
                row['instance_number_0'] = instance_numbers[max((total_len * 3) // 5,1)]
                row['instance_number_1'] = instance_numbers[max((total_len * 3) // 5,1)]
                row['instance_number_2'] = instance_numbers[max((total_len * 3) // 5,1)]
                row['instance_number_3'] = instance_numbers[max((total_len * 3) // 5,1)]
                row['instance_number_4'] = instance_numbers[max((total_len * 3) // 5,1)]
            elif missing_level == 'L4/L5':
                row['instance_number_0'] = instance_numbers[max((total_len * 4) // 5,1)]
                row['instance_number_1'] = instance_numbers[max((total_len * 4) // 5,1)]
                row['instance_number_2'] = instance_numbers[max((total_len * 4) // 5,1)]
                row['instance_number_3'] = instance_numbers[max((total_len * 4) // 5,1)]
                row['instance_number_4'] = instance_numbers[max((total_len * 4) // 5,1)]
            elif missing_level == 'L5/S1':
                row['instance_number_0'] = instance_numbers[total_len-1]
                row['instance_number_1'] = instance_numbers[total_len-1]
                row['instance_number_2'] = instance_numbers[total_len-1]
                row['instance_number_3'] = instance_numbers[total_len-1]
                row['instance_number_4'] = instance_numbers[total_len-1]

            # Append the filled row
            filled_rows.append(row)

    # Combine the new filled rows with the original dataframe
    filled_df = pd.concat([grouped_df_axial, pd.DataFrame(filled_rows)], ignore_index=True)
    return filled_df


grouped_df_axial = fill_missing_levels(grouped_df_axial, all_df)


In [109]:
grouped_df_axial=grouped_df_axial[grouped_df_axial['label'].notna()]

In [110]:
grouped_df_axial.reset_index(drop=True,inplace=True)

In [114]:
# DataFrameを保存するパス
save_path = '/content/drive/MyDrive/RSNA_csv/grouped_df_axial_rev.pkl'

# DataFrameをpkl形式で保存
grouped_df_axial.to_pickle(save_path)

print(f"DataFrame has been saved to {save_path}")

DataFrame has been saved to /content/drive/MyDrive/RSNA_csv/grouped_df_axial_rev.pkl


# 画像認識モデル学習部分

In [115]:
with open("/content/drive/MyDrive/RSNA_csv/grouped_df_axial_rev.pkl", 'rb') as f:
  grouped_df_axial=pickle.load(f)

In [117]:
unique_studies = grouped_df_axial['study_id'].unique()
study_mapping = {study: (i % 5) + 1 for i, study in enumerate(unique_studies)}

# Map the numbers back to the original DataFrame
grouped_df_axial['series_description2'] = grouped_df_axial['study_id'].map(study_mapping)


In [118]:
grouped_df_axial_left=grouped_df_axial[grouped_df_axial['direction']==0]
grouped_df_axial_right=grouped_df_axial[grouped_df_axial['direction']==1]

In [119]:
tdf2 = grouped_df_axial_right[grouped_df_axial_right['series_description2'] != fold]
vdf2 = grouped_df_axial_right[grouped_df_axial_right['series_description2'] == fold]

In [120]:
import albumentations as A

AUG_PROB = 0.75
transforms_train = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=AUG_PROB),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        # A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=AUG_PROB),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=AUG_PROB),

    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),
  #  A.CoarseDropout(max_holes=15, max_height=30, max_width=30, min_holes=1, min_height=8, min_width=8, p=AUG_PROB),
])

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [121]:
patch_size=224

In [122]:
import torch.nn.functional as F
class ViT_T1_Dataset(Dataset):
    def __init__(self, df, UNet, VALID=False, P=patch_size, alpha=0,transform=None):
        self.data = df
        self.UNet = UNet
        self.VALID = VALID
        self.P = P
        self.alpha = alpha
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = np.zeros((5, self.P, self.P), dtype=np.float32)
        non_zero_slice=[]
        for i in range(5):

          sample = '/content/train_images/'
          sample = sample+str(self.data.iloc[index]['study_id'])+'/'+str(self.data.iloc[index]['series_id'])+'/'+str(self.data.iloc[index][f'instance_number_{i}'])+'.dcm'

          image = pydicom.dcmread(sample).pixel_array
          H,W = image.shape
          # centers = torch.as_tensor([x for x in row[coor]]).view(5,2).float()
          # By plane resizing I've been distorting the proportions
          if H > W:
              d = W
              h = (H - d)//2
              image = image[h:h+d]
              # centers[:,1] -= h
              H = W
          elif H < W:
              d = H
              w = (W - d)//2
              image = image[:,w:w+d]
              # centers[:,0] -= w
              W = H
          image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
          image = torch.as_tensor(image/(np.max(image))).unsqueeze(0).unsqueeze(0).float().to(device)
          # x[i,...] = torch.as_tensor(image/(np.max(image))).cpu().numpy()

          # OUT = 0
          # with torch.no_grad():
                  # for rot in [0,1,2,3]:
                          # OUT += torch.rot90(self.UNet(torch.rot90(image,rot,dims=[-2, -1])),-rot,dims=[-2, -1])
          OUT=self.UNet(image)
          OUT = (OUT > TH)[0]
          c = (OUT.unsqueeze(1)*idx_map[0]).view(2,2,PATCH_SIZE*PATCH_SIZE).sum(-1)
          d = OUT.view(2,PATCH_SIZE*PATCH_SIZE).sum(-1)
          m = d > 0
          c[m] = (c[m]/(d[m]).unsqueeze(-1)).long()
          c[~m] = self.P # I have to find a better solution

          image_slices = []

          for xy in c:
            y_start = max(0, xy[1] - self.P // 2)
            y_end = min(512,xy[1] + self.P - self.P // 2)
            x_start = max(0, xy[0] - self.P // 2)
            x_end = min(512,xy[0] + self.P - self.P // 2)

    # スライスが有効なサイズを持つか確認
            if (y_end - y_start == self.P) and (x_end - x_start == self.P):
              slice_img = image[0, 0, y_start:y_end, x_start:x_end]
              non_zero_slice.append(slice_img)
              image_slices.append(slice_img)
            else:
              zero_slice = torch.zeros((self.P, self.P), device=image.device)
              image_slices.append(zero_slice)
              print(f"Skipped slice due to incorrect size: {(y_end - y_start, x_end - x_start)}")
              print(f"Slice coordinates: {(xy[1], xy[0])}")

# スライスがあればスタックする
          if image_slices:
            try:
              image = torch.stack(image_slices)
            except RuntimeError as e:
              print(f"Error: {e}")
          else:
            print("No valid slices available for stacking.")

          # if not self.VALID: image = augment_image(image,self.alpha)
          x[i,...]=image[self.data.iloc[index]['direction']].cpu().numpy()

        for i in range(5):
          if (x[i,...].sum() == 0) and (len(non_zero_slice)>0):
            x[i,...] = non_zero_slice[0].cpu().numpy()
          else:
            pass
            #nothing
        if self.transform is not None:
          x = self.transform(image=x)['image']
        x=torch.as_tensor(x).float()
        # x = F.interpolate(x.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
        m=torch.as_tensor(np.ones(3)).bool()
        label = torch.as_tensor(labels[self.data.iloc[index]['label']])

        return [x.to(device),m.to(device)],[label.to(device),m.to(device)]


In [123]:
def myLoss(preds,target):
    target,mask = target
    # target=target[mask]
    # preds = preds[mask.view(-1)]
    return nn.CrossEntropyLoss(weight=torch.as_tensor([1.,2.,4.]).to(device))(preds+1e-12,target)

In [124]:
import timm
class ViT(nn.Module):
    def __init__(self, num_classes):
        super(ViT, self).__init__()
        # ここにViTのモデルアーキテクチャを実装
        self.vit = timm.create_model('eva02_base_patch14_224', pretrained=True, num_classes=num_classes,in_chans=5,features_only=False,global_pool='avg')
        # self.vit.features.conv0=nn.Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # self.new_model=nn.Sequential(*self.layer)
    def forward(self, x):
        # ここに順伝播の処理を実装
        x,mask=x
        return self.vit(x)

In [125]:
from fastai.callback.core import Callback

class SaveModelCallback(Callback):
    def __init__(self, every_epoch=False, path='models', fname='model',with_opt=False):
        self.every_epoch = every_epoch
        self.path = path
        self.fname = fname
        self.with_opt=with_opt

    def after_epoch(self):
        # エポックごとにモデルを保存する
        if self.every_epoch:
            self.learn.save(f'{self.path}/{self.fname}_ep_{self.epoch}')

save_model_cb = SaveModelCallback(with_opt=True,every_epoch=True, path='/content/drive/MyDrive/RSNA_csv', fname=f"enb_axial_r_dd5_con_p_224_f_{fold}")

In [126]:
if 1:
    seed_everything(SEED)
    with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+"1"+"_SS"+".pkl", 'rb') as f:
      UNet=pickle.load(f)
    tds = ViT_T1_Dataset(tdf2,UNet,transform=transforms_train)
    vds = ViT_T1_Dataset(vdf2,UNet)
    tdl = torch.utils.data.DataLoader(tds, batch_size=16, shuffle=True, drop_last=True)
    vdl = torch.utils.data.DataLoader(vds, batch_size=16, shuffle=False)

    dls = DataLoaders(tdl,vdl)

    n_iter = len(tds)//INF['BS']

    model = ViT(num_classes=3)
    model.to(device)
    learn = Learner(
        dls,
        model,
        lr=INF['LR'],
        loss_func=myLoss,
        cbs=[
            save_model_cb,
            GradientClip,
            ShowGraphCallback(),
            # alpha_cb
        ]
    )
    learn.fit_one_cycle(INF['EPOCHS'])
    # with open('/content/drive/MyDrive/RSNA_csv/'+"VIT_"+str(fold)+".pkl", 'wb') as f:
      # pickle.dump(model, f)

epoch,train_loss,valid_loss,time


Skipped slice due to incorrect size: (tensor(224, device='cuda:0'), tensor(208, device='cuda:0'))
Slice coordinates: (tensor(123, device='cuda:0'), tensor(416, device='cuda:0'))
Skipped slice due to incorrect size: (tensor(224, device='cuda:0'), tensor(124, device='cuda:0'))
Slice coordinates: (tensor(279, device='cuda:0'), tensor(500, device='cuda:0'))
Skipped slice due to incorrect size: (tensor(207, device='cuda:0'), tensor(178, device='cuda:0'))
Slice coordinates: (tensor(417, device='cuda:0'), tensor(446, device='cuda:0'))


KeyboardInterrupt: 